In [1]:
import os
import json
import regex as re
from tqdm.auto import tqdm
import math
import pandas as pd
import string

import numpy as np

from pyvi.ViTokenizer import tokenize
import nltk
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=16)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [8]:
data_root = 'data'
sliding_wiki_name = "wikipedia_20220620_cleaned_v2.csv"

## Create sliding wiki

In [12]:
_WORD_SPLIT = re.compile("([.,!?\"/':;)(])")
STOP_WORDS = "\" \' [ ] . , ! : ; ?".split(" ")

def basic_tokenizer(sentence):
    """Very basic tokenizer: split the sentence into a list of tokens."""
    words = []
    for space_separated_fragment in sentence.strip().split():
        words.extend(_WORD_SPLIT.split(space_separated_fragment))
        # return [w.lower() for w in words if w not in stop_words and w != '' and w != ' ']
    return [w.lower() for w in words if w != '' and w != ' ' and w not in string.punctuation]

def remove_appending_title(text,title):
    return text.replace(f"{title}\n\n{title}",f"{title} ")

def create_sliding_window(text, size=256, overlap=32):
    actual_size = size - overlap
    windows = []
    n_windows = math.ceil(len(text)/actual_size)
    for i in range(n_windows):
        windows.append(" ".join(text[i*actual_size:i*actual_size + size]))
    return windows

In [4]:
all_titles = []
all_texts = []
all_bm25_texts = []
with open(data_root + "/wikipedia_20220620_cleaned.jsonl") as f:
    for i, line in tqdm(enumerate(f)):
        try:
            x = json.loads(line)
            text = remove_appending_title(x["text"], x["title"])
            # print(y, "|", text[:50],"\n")
            text = text.split(" ")
            sliding_windows = create_sliding_window(text)
            # bm25_windows = [" ".join(basic_tokenizer(w)) for w in sliding_windows]
            all_texts.extend(sliding_windows)
            # all_bm25_texts.extend(bm25_windows)
            all_titles.extend([x['title'],]*len(sliding_windows))
        except:
            print(line)

0it [00:00, ?it/s]

{"id": "2913123", "url": "https://vi.wikipedia.org/wiki?curid=2913123", "title": "Dusponera atrisignalis", "text": "Dusponera atrisignalis\n\nDusponera atrisignalis là một loài bướm đêm trong họ Noctuidae.\n\n", "t


In [5]:
df = pd.DataFrame()
df["title"] = all_titles
df["text"] = all_texts

In [6]:
df["bm25_text"] = df["text"].parallel_apply(lambda x: " ".join(basic_tokenizer(x)))

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [9]:
df = df.iloc[1:, :]

In [10]:
df

,title,text,bm25_text
1,Internet Society,Internet Society hay ISOC là một tổ chức quốc...,internet society hay isoc là một tổ chức quốc ...
2,Tiếng Việt,"Tiếng Việt , cũng gọi là tiếng Việt Nam hay Vi...",tiếng việt cũng gọi là tiếng việt nam hay việt...
3,Tiếng Việt,"hệ thống thanh điệu phát triển cao hơn, hệ thố...",hệ thống thanh điệu phát triển cao hơn hệ thốn...
4,Tiếng Việt,tiếp xúc Hán – Việt thành 2 giai đoạn chính: \...,tiếp xúc hán – việt thành 2 giai đoạn chính bu...
5,Tiếng Việt,"thêm hàng loạt các yếu tố Hán–Việt. Như là ""ch...",thêm hàng loạt các yếu tố hán–việt như là chủ ...
...,...,...,...
1469631,Aporophoba subaustralis,Aporophoba subaustralis là một loài bướm đêm ...,aporophoba subaustralis là một loài bướm đêm t...
1469632,Dunira sarconia,Dunira sarconia là một loài bướm đêm trong họ...,dunira sarconia là một loài bướm đêm trong họ ...
1469633,Aporophyla aethiops,Aporophyla aethiops là một loài bướm đêm tron...,aporophyla aethiops là một loài bướm đêm trong...
1469634,Dunira scitula,Dunira scitula là một loài bướm đêm trong họ ...,dunira scitula là một loài bướm đêm trong họ e...


In [11]:
df.to_csv(data_root + '/' + sliding_wiki_name, index=False)

In [12]:
print('Done')

Done


## Create BM25 database

In [9]:
df = pd.read_csv(data_root + '/' + sliding_wiki_name)

In [10]:
df.isna().sum()

title        1
text         0
bm25_text    7
dtype: int64

In [11]:
df = df.dropna()

In [12]:
df.isna().sum()

title        0
text         0
bm25_text    0
dtype: int64

In [14]:
def strip_context(text): 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    return text

dict_map = dict({})  
def word_tokenize(text): 
    global dict_map 
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if dict_map.get(w, None) is None: 
            dict_map[w] = ' '.join(nltk.word_tokenize(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm 
 

def post_process(x):
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n"," ")
    x = "".join([i for i in x if i not in string.punctuation])
    return x

In [14]:
df['bm25_text'] = df['bm25_text'].apply(lambda x: x.lower()).parallel_apply(post_process)

In [15]:
corpus = [x.split() for x in df['bm25_text'].values]

In [16]:
print(corpus[0])

['internet', 'society', 'hay', 'isoc', 'là', 'một', 'tổ', 'chức', 'quốc', 'tế', 'hoạt', 'động', 'phi', 'lợi', 'nhuận', 'phi', 'chính', 'phủ', 'và', 'bao', 'gồm', 'các', 'thành', 'viên', 'có', 'trình', 'độ', 'chuyên', 'ngành', 'tổ', 'chức', 'này', 'chú', 'trọng', 'đến', 'tiêu', 'chuẩn', 'giáo', 'dục', 'và', 'các', 'vấn', 'đề', 'về', 'chính', 'sách', 'với', 'trên', '145', 'tổ', 'chức', 'thành', 'viên', 'và', '65', '000', 'thành', 'viên', 'cá', 'nhân', 'isoc', 'bao', 'gồm', 'những', 'con', 'người', 'cụ', 'thể', 'trong', 'cộng', 'đồng', 'internet', 'mọi', 'chi', 'tiết', 'có', 'thể', 'tìm', 'thấy', 'tại', 'website', 'của', 'isoc', 'internet', 'society', 'nằm', 'ở', 'gần', 'thủ', 'đô', 'washington', 'dc', 'hoa', 'kỳ', 'và', 'geneva', 'thụy', 'sĩ', 'số', 'hội', 'viên', 'của', 'nó', 'bao', 'gồm', 'hơn', '145', 'tổ', 'chức', 'thành', 'viên', 'và', 'hơn', '65', '000', 'cá', 'nhân', 'thành', 'viên', 'còn', 'có', 'thể', 'tự', 'lập', 'một', 'chi', 'nhánh', 'của', 'tổ', 'chức', 'tùy', 'theo', 'vị', 

In [17]:
dictionary = Dictionary(corpus)

In [19]:
bm25_model = OkapiBM25Model(dictionary=dictionary)

In [20]:
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]

In [23]:
bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary), normalize_queries=False, normalize_documents=False)

In [24]:
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')

In [25]:
dictionary.save("dictionary")
tfidf_model.save("tfidf")
bm25_index.save("bm25_index")

## Test

In [38]:
class BM25Model:
    def __init__(self, dictionary, tfidf_model, bm25_index) -> None:
        self.dictionary = dictionary
        self.tfidf_model = tfidf_model
        self.bm25_index = bm25_index
        
    def get_topk_stage1(self, query, topk=100):
        tokenized_query = query.split()
        tfidf_query = self.tfidf_model[self.dictionary.doc2bow(tokenized_query)]
        scores = self.bm25_index[tfidf_query]
        top_n = np.argsort(scores)[::-1][:topk]
        return top_n, scores[top_n]

In [18]:
def preprocess(x, max_length=-1, remove_puncts=False):
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n", " ")
    if remove_puncts:
        x = "".join([i for i in x if i not in string.punctuation])
    if max_length > 0:
        x = " ".join(x.split()[:max_length])
    return x

def get_answer_e2e(bm25_model, question, df_wiki_windows):
    #Bm25 retrieval for top200 candidates
    query = preprocess(question).lower()
    top_n, bm25_scores = bm25_model.get_topk_stage1(query, topk=200)
    titles = [preprocess(df_wiki_windows.title.values[i]) for i in top_n]
    texts = [preprocess(df_wiki_windows.text.values[i]) for i in top_n]
    
    return titles, texts

In [32]:
checkpoint_path = '.'
dictionary = Dictionary.load(checkpoint_path + '/dictionary')
tfidf_model = SparseMatrixSimilarity.load(checkpoint_path + '/tfidf')
bm25_index = TfidfModel.load(checkpoint_path + '/bm25_index')

In [39]:
bm25_model = BM25Model(dictionary, tfidf_model, bm25_index)

In [10]:
df_wiki_windows = pd.read_csv("./data/wikipedia_20220620_cleaned_v2.csv")

In [55]:
df_wiki_windows[df_wiki_windows['title'] == '2670 Chuvashia']

,title,text,bm25_text
526817,2670 Chuvashia,2670 Chuvashia (1977 PW1) là một tiểu hành ti...,2670 chuvashia 1977 pw1 là một tiểu hành tinh ...


In [40]:
titles, texts = get_answer_e2e(bm25_model, "Ai là tác giả Lord of the Rings", df_wiki_windows)

In [48]:
import textwrap 
wrapper = textwrap.TextWrapper(width=120) 

In [56]:
for text in texts:
    text = wrapper.fill(text)
    print(text)
    print()

2670 Chuvashia ( 1977 PW1 ) là một tiểu hành tinh vành đai chính được phát hiện ngày 14 tháng 8 năm 1977 bởi Chernykh ,
N . ở Đài vật lý thiên văn Crimean . Nó được đặt theo tên the Russian republic thuộc Chuvashia . == Liên kết ngoài . ==
BULLET : : : :- JPL Small-Body Database Browser ngày 2670 Chuvashia

xe đi vào rạp . Vào cảnh cuối chương trình Inspector lên xe chạy bỏ Báo Hồng ở lại . === Laugh track . === Như nhiều
hoạt hình Ngày thứ Bảy bấy giờ , " The Pink Panther Show " chứa những đoạn âm thanh cười được thu sẵn phát lồng vào phim
( Laugh track ) , về sau bị gỡ bỏ . Boomerang vẫn còn chiếu phiên bản với những tiếng cười này , và chúng vẫn còn thấy ở
các phiên bản : Tiếng Tây Ban Nha , Tiếng Bồ Đào Nha , Boomerangs , the Pháp-based Gulli , và Phần Lan TV 4 và TV 6 .
=== Các phiên bản . === Suốt 11 năm trên nhiều hệ thống kênh truyền hình , " The Pink Panther Show " có nhiều tên :
BULLET : : : :- " The Pink Panther Show " ( 1969–1970 ) BULLET : : : :- " The Pink Panther Meets t

In [57]:
exit()

: 